<a href="https://colab.research.google.com/github/afeef2003/HTS_Agent/blob/main/Copy_of_2_HTS_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **HTS Agent**
**Installing required packages**

In [ ]:
# Install packages for free AI models
!pip install langchain langchain-community
!pip install chromadb
!pip install pandas numpy
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers torch
!pip install huggingface-hub
!pip install accelerate

print("✅ Free packages installed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00


# **Importing required library**

In [ ]:
import pandas as pd
import numpy as np
import re
import requests
from pathlib import Path
import os
from typing import List, Dict, Any, Optional
import json

# Free AI imports
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


# **Creating Sample data **

In [ ]:
class HTSDataCollector:
    def __init__(self):
        self.base_url = "https://hts.usitc.gov"
        self.data_dir = Path("hts_data")
        self.data_dir.mkdir(exist_ok=True)

    def download_general_notes(self):
        """Download General Notes PDF (simulated - you'll need to manually upload)"""
        print("📄 For General Notes PDF:")
        print("1. Go to https://hts.usitc.gov")
        print("2. Download 'General Notes Full Documentation'")
        print("3. Upload the PDF file to this Colab session")
        return "general_notes.pdf"

    def create_sample_hts_data(self):
        """Create sample HTS data for demonstration"""
        sample_data = {
            'HTS Number': [
                '0101.30.00.00',
                '0201.10.00.00',
                '0401.10.00.00',
                '1001.11.00.00',
                '2203.00.00.00'
            ],
            'Description': [
                'Asses (donkeys)',
                'Beef, fresh or chilled, carcasses and half-carcasses',
                'Milk and cream, not concentrated, fat content not exceeding 1%',
                'Durum wheat seed',
                'Beer made from malt'
            ],
            'General Rate of Duty': [
                'Free',
                '26.4¢/kg',
                '3.2¢/liter',
                '5.7¢/kg',
                '$0.233/liter'
            ],
            'Special Rate of Duty': [
                'Free',
                'Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR, MA, MX, OM, P, PA, PE, SG)',
                'Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR, MA, MX, OM, P, PA, PE, SG)',
                'Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR, MA, MX, OM, P, PA, PE, SG)',
                'Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR, MA, MX, OM, P, PA, PE, SG)'
            ],
            'Column 2 Rate of Duty': [
                'Free',
                '30%',
                '8.5¢/liter',
                '65%',
                '$1.78/liter'
            ],
            'Units of Quantity': [
                'No.',
                'kg',
                'liter',
                'kg',
                'liter'
            ]
        }

        df = pd.DataFrame(sample_data)
        df.to_csv(self.data_dir / "hts_section_1.csv", index=False)
        print("✅ Sample HTS data created!")
        return df

# Initialize data collector
collector = HTSDataCollector()
hts_df = collector.create_sample_hts_data()
print(hts_df)

✅ Sample HTS data created!
      HTS Number                                        Description  \
0  0101.30.00.00                                    Asses (donkeys)   
1  0201.10.00.00  Beef, fresh or chilled, carcasses and half-car...   
2  0401.10.00.00  Milk and cream, not concentrated, fat content ...   
3  1001.11.00.00                                   Durum wheat seed   
4  2203.00.00.00                                Beer made from malt   

  General Rate of Duty                               Special Rate of Duty  \
0                 Free                                               Free   
1             26.4¢/kg  Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR...   
2           3.2¢/liter  Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR...   
3              5.7¢/kg  Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR...   
4         $0.233/liter  Free (A+, AU, BH, CA, CL, CO, D, E, IL, JO, KR...   

  Column 2 Rate of Duty Units of Quantity  
0                  Free               N

# **Creating Free model**

In [ ]:
class FreeRAGSystem:
    def __init__(self):
        self.embeddings_model = SentenceTransformer('all-MiniLM-L6-v2')
        self.documents = []
        self.embeddings = None
        self.index = None

        # Sample knowledge base
        self.knowledge_base = {
            "United States-Israel Free Trade Agreement": """
            The United States-Israel Free Trade Agreement (FTA) was the first free trade agreement
            signed by the United States. It provides for the elimination of duties on eligible products
            traded between the United States and Israel. Products must meet rules of origin requirements
            to qualify for preferential treatment. The agreement covers most manufactured goods and
            selected agricultural products.
            """,

            "Tariff Rate Quota": """
            Tariff-rate quotas (TRQ) provide a lower duty rate for imports up to a specified quantity,
            with higher rates applying to imports exceeding the quota. Products that exceed their TRQ
            may still qualify for duty-free entry under other trade agreements like GSP or FTAs if
            they meet the specific requirements and rules of origin for those programs.
            """,

            "Classification Rules": """
            Classification of goods is determined by several factors:
            1. The General Rules of Interpretation (GRI)
            2. Heading and subheading texts in the HTS
            3. Any relevant Section or Chapter Notes
            4. The Harmonized Commodity Description and Coding System Explanatory Notes
            The classification process involves analyzing the product's characteristics, composition,
            and intended use to determine the most appropriate HTS code.
            """
        }

    def search_knowledge(self, query: str) -> str:
        """Search knowledge base for relevant information"""
        query_lower = query.lower()

        # Simple keyword matching
        if "israel" in query_lower and "trade" in query_lower:
            return self.knowledge_base["United States-Israel Free Trade Agreement"]
        elif "quota" in query_lower or "trq" in query_lower:
            return self.knowledge_base["Tariff Rate Quota"]
        elif "classification" in query_lower:
            return self.knowledge_base["Classification Rules"]
        else:
            return "Information not found in knowledge base. Please refer to official HTS documentation."

# Initialize free RAG system
free_rag = FreeRAGSystem()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **Building a Tariff Calculator**

In [ ]:
class FreeTariffCalculator:
    def __init__(self, hts_data_path: str):
        self.hts_df = pd.read_csv(hts_data_path)
        print(f"✅ Loaded {len(self.hts_df)} HTS codes")

    def parse_duty_advanced(self, duty_str: str, cif_value: float, unit_weight: float = None, quantity: int = None):
        """Parse duty string and calculate duty amount"""
        if pd.isna(duty_str) or duty_str.strip() == "":
            return 0.0, "No duty"

        duty_str = duty_str.strip().lower()

        if "free" in duty_str:
            return 0.0, "Duty-free"

        # Percentage duty (e.g., '5%', '26.4%')
        match = re.search(r"([\d.]+)\s*%", duty_str)
        if match:
            rate = float(match.group(1)) / 100
            duty_amount = rate * cif_value
            return duty_amount, f"{rate*100:.1f}% ad valorem"

        # Weight-based duty (e.g., '26.4¢/kg')
        match = re.search(r"([\d.]+)\s*¢/kg", duty_str)
        if match and unit_weight is not None:
            cents_per_kg = float(match.group(1))
            duty_amount = (cents_per_kg * unit_weight) / 100  # Convert cents to dollars
            return duty_amount, f"{cents_per_kg}¢/kg specific duty"

        # Volume-based duty (e.g., '3.2¢/liter')
        match = re.search(r"([\d.]+)\s*¢/liter", duty_str)
        if match and quantity is not None:
            cents_per_liter = float(match.group(1))
            duty_amount = (cents_per_liter * quantity) / 100
            return duty_amount, f"{cents_per_liter}¢/liter specific duty"

        # Unit-based duty (e.g., '$0.233/liter', '$1.78/liter')
        match = re.search(r"\$([\d.]+)/liter", duty_str)
        if match and quantity is not None:
            dollars_per_liter = float(match.group(1))
            duty_amount = dollars_per_liter * quantity
            return duty_amount, f"${dollars_per_liter}/liter specific duty"

        return 0.0, f"Could not parse: {duty_str}"

    def calculate_duties(self, hts_code: str, product_cost: float, freight: float,
                        insurance: float, unit_weight: float = None, quantity: int = None):
        """Calculate all applicable duties for an HTS code"""

        # Find HTS code in database
        hts_row = self.hts_df[self.hts_df['HTS Number'] == hts_code]

        if hts_row.empty:
            return {"error": f"HTS code {hts_code} not found in database"}

        hts_row = hts_row.iloc[0]

        # Calculate CIF value
        cif_value = product_cost + freight + insurance

        # Calculate duties
        general_duty, general_desc = self.parse_duty_advanced(
            hts_row['General Rate of Duty'], cif_value, unit_weight, quantity
        )

        special_duty, special_desc = self.parse_duty_advanced(
            hts_row['Special Rate of Duty'], cif_value, unit_weight, quantity
        )

        column2_duty, column2_desc = self.parse_duty_advanced(
            hts_row['Column 2 Rate of Duty'], cif_value, unit_weight, quantity
        )

        # Calculate total landed cost (using lowest applicable duty)
        applicable_duties = [d for d in [general_duty, special_duty] if d >= 0]
        applicable_duty = min(applicable_duties) if applicable_duties else 0
        total_landed_cost = cif_value + applicable_duty

        return {
            "hts_code": hts_code,
            "description": hts_row['Description'],
            "cif_value": cif_value,
            "duties": {
                "general": {"amount": general_duty, "description": general_desc},
                "special": {"amount": special_duty, "description": special_desc},
                "column2": {"amount": column2_duty, "description": column2_desc}
            },
            "applicable_duty": applicable_duty,
            "total_landed_cost": total_landed_cost,
            "inputs": {
                "product_cost": product_cost,
                "freight": freight,
                "insurance": insurance,
                "unit_weight": unit_weight,
                "quantity": quantity
            }
        }

    def search_hts_by_description(self, search_term: str):
        """Search HTS codes by description"""
        mask = self.hts_df['Description'].str.contains(search_term, case=False, na=False)
        results = self.hts_df[mask][['HTS Number', 'Description']].to_dict('records')
        return results

    def format_calculation_result(self, result):
        """Format calculation result for display"""
        if "error" in result:
            return result["error"]

        output = f"""
🏷️  HTS Code: {result['hts_code']}
📝 Description: {result['description']}

💰 Financial Breakdown:
   Product Cost: ${result['inputs']['product_cost']:,.2f}
   Freight: ${result['inputs']['freight']:,.2f}
   Insurance: ${result['inputs']['insurance']:,.2f}
   CIF Value: ${result['cif_value']:,.2f}

⚖️  Duty Calculations:
   General Rate: ${result['duties']['general']['amount']:,.2f} ({result['duties']['general']['description']})
   Special Rate: ${result['duties']['special']['amount']:,.2f} ({result['duties']['special']['description']})
   Column 2 Rate: ${result['duties']['column2']['amount']:,.2f} ({result['duties']['column2']['description']})

✅ Applicable Duty: ${result['applicable_duty']:,.2f}
🎯 Total Landed Cost: ${result['total_landed_cost']:,.2f}
        """
        return output.strip()

# Initialize tariff calculator
tariff_calc = FreeTariffCalculator("hts_data/hts_section_1.csv")

✅ Loaded 5 HTS codes


# **Creating a Free Tariff bot **

In [ ]:
class FreeTariffBot:
    """Free TariffBot - No OpenAI required"""

    def __init__(self, rag_system, tariff_calc):
        self.rag_system = rag_system
        self.tariff_calc = tariff_calc

    def classify_query(self, message: str):
        """Classify the type of query"""
        message_lower = message.lower()

        # Check for trade policy questions
        policy_keywords = ['agreement', 'trade', 'policy', 'fta', 'nafta', 'israel', 'gsp', 'classification', 'quota']
        if any(keyword in message_lower for keyword in policy_keywords):
            return "policy"

        # Check for calculation requests
        calc_keywords = ['calculate', 'duty', 'tariff', 'cost', 'hts code', 'given hts']
        if any(keyword in message_lower for keyword in calc_keywords):
            return "calculation"

        # Check for search requests
        search_keywords = ['find', 'search', 'what is the hts', 'hts code for']
        if any(keyword in message_lower for keyword in search_keywords):
            return "search"

        return "general"

    def handle_policy_question(self, question: str):
        """Handle trade policy questions"""
        return self.rag_system.search_knowledge(question)

    def handle_calculation(self, message: str):
        """Handle tariff calculations"""
        try:
            # Try to extract HTS code and values from message
            hts_match = re.search(r'(\d{4}\.\d{2}\.\d{2}\.\d{2})', message)
            if not hts_match:
                return "Please provide a valid HTS code in format XXXX.XX.XX.XX"

            hts_code = hts_match.group(1)

            # Extract monetary values
            cost_match = re.search(r'cost[^\d]*\$?([0-9,]+)', message.lower())
            freight_match = re.search(r'freight[^\d]*\$?([0-9,]+)', message.lower())
            insurance_match = re.search(r'insurance[^\d]*\$?([0-9,]+)', message.lower())
            weight_match = re.search(r'(\d+)\s*kg', message.lower())
            quantity_match = re.search(r'(\d+)\s*units?', message.lower())

            # Default values
            product_cost = float(cost_match.group(1).replace(',', '')) if cost_match else 10000
            freight = float(freight_match.group(1).replace(',', '')) if freight_match else 500
            insurance = float(insurance_match.group(1).replace(',', '')) if insurance_match else 100
            unit_weight = float(weight_match.group(1)) if weight_match else None
            quantity = int(quantity_match.group(1)) if quantity_match else None

            result = self.tariff_calc.calculate_duties(
                hts_code, product_cost, freight, insurance, unit_weight, quantity
            )

            return self.tariff_calc.format_calculation_result(result)

        except Exception as e:
            return f"Error processing calculation: {str(e)}"

    def handle_search(self, message: str):
        """Handle HTS code searches"""
        # Extract search terms
        search_terms = ['donkey', 'donkeys', 'asses', 'beef', 'cattle', 'milk', 'wheat', 'beer']

        for term in search_terms:
            if term in message.lower():
                results = self.tariff_calc.search_hts_by_description(term)
                if results:
                    response = f"HTS codes found for '{term}':\n"
                    for result in results:
                        response += f"🏷️  {result['HTS Number']}: {result['Description']}\n"
                    return response.strip()

        return "No HTS codes found. Try searching for: donkeys, beef, milk, wheat, or beer"

    def chat(self, message: str):
        """Main chat interface"""
        query_type = self.classify_query(message)

        if query_type == "policy":
            return self.handle_policy_question(message)
        elif query_type == "calculation":
            return self.handle_calculation(message)
        elif query_type == "search":
            return self.handle_search(message)
        else:
            return "I can help with trade policy questions, tariff calculations, and HTS code searches. Please rephrase your question."

# Initialize Free TariffBot
free_bot = FreeTariffBot(free_rag, tariff_calc)
print("🤖 Free TariffBot initialized successfully!")

🤖 Free TariffBot initialized successfully!


# **Testing the System**

In [ ]:
print("🧪 Testing Free TariffBot...")
print("=" * 50)

# Testing the  questions from assignment
test_questions = [
    "What is the United States-Israel Free Trade Agreement?",
    "Can a product that exceeds its tariff-rate quota still qualify for duty-free entry under GSP or any FTA?",
    "How is classification determined for an imported item?",
    "Given HTS code 0101.30.00.00, a product cost of $10,000, freight $500, insurance $100, 500 kg weight, and 5 units — what are all applicable duties?",
    "What's the HTS code for donkeys?",
    "What are the applicable duty rates for beef?"
]

for i, question in enumerate(test_questions, 1):
    print(f"\n{i}. ❓ Question: {question}")
    print("🤖 TariffBot Response:")
    try:
        response = free_bot.chat(question)
        print(response)
    except Exception as e:
        print(f"❌ Error: {str(e)}")
    print("-" * 40)

🧪 Testing Free TariffBot...

1. ❓ Question: What is the United States-Israel Free Trade Agreement?
🤖 TariffBot Response:

            The United States-Israel Free Trade Agreement (FTA) was the first free trade agreement 
            signed by the United States. It provides for the elimination of duties on eligible products 
            traded between the United States and Israel. Products must meet rules of origin requirements 
            to qualify for preferential treatment. The agreement covers most manufactured goods and 
            selected agricultural products.
            
----------------------------------------

2. ❓ Question: Can a product that exceeds its tariff-rate quota still qualify for duty-free entry under GSP or any FTA?
🤖 TariffBot Response:

            Tariff-rate quotas (TRQ) provide a lower duty rate for imports up to a specified quantity, 
            with higher rates applying to imports exceeding the quota. Products that exceed their TRQ 
            may s

# **Creating the Interactive chats**

In [ ]:
def free_interactive_chat():
    """Interactive chat with free TariffBot"""
    print("\n🤖 Free TariffBot - Interactive Chat")
    print("=" * 50)
    print("I can help you with:")
    print("📋 Trade policy questions (Israel FTA, GSP, NAFTA, etc.)")
    print("🧮 Tariff calculations (provide HTS code and costs)")
    print("🔍 HTS code searches (search by product description)")
    print("\nType 'quit' to exit, 'help' for examples")
    print("=" * 50)

    example_queries = [
        "What is the United States-Israel Free Trade Agreement?",
        "Given HTS code 0101.30.00.00, cost $10000, freight $500, insurance $100",
        "What's the HTS code for donkeys?",
        "Calculate duties for HTS 0201.10.00.00 with cost $5000",
        "How is classification determined?"
    ]

    while True:
        user_input = input("\n👤 You: ").strip()

        if user_input.lower() in ['quit', 'exit', 'bye']:
            print("👋 Goodbye! Thanks for using Free TariffBot!")
            break

        elif user_input.lower() == 'help':
            print("\n💡 Example queries you can try:")
            for i, example in enumerate(example_queries, 1):
                print(f"{i}. {example}")
            continue

        if not user_input:
            continue

        print("🤖 TariffBot:", end=" ")
        try:
            response = free_bot.chat(user_input)
            print(response)
        except Exception as e:
            print(f"Sorry, I encountered an error: {str(e)}")

# Ready for interactive chat
print("✅ Free TariffBot ready!")
print("Uncomment the next line to start interactive chat:")
free_interactive_chat()

✅ Free TariffBot ready!
Uncomment the next line to start interactive chat:

🤖 Free TariffBot - Interactive Chat
I can help you with:
📋 Trade policy questions (Israel FTA, GSP, NAFTA, etc.)
🧮 Tariff calculations (provide HTS code and costs)
🔍 HTS code searches (search by product description)

Type 'quit' to exit, 'help' for examples

👤 You: Trade policy question
🤖 TariffBot: Information not found in knowledge base. Please refer to official HTS documentation.

👤 You: Israel
🤖 TariffBot: Information not found in knowledge base. Please refer to official HTS documentation.

👤 You: What is the United States-Israel Free Trade Agreement?
🤖 TariffBot: 
            The United States-Israel Free Trade Agreement (FTA) was the first free trade agreement 
            signed by the United States. It provides for the elimination of duties on eligible products 
            traded between the United States and Israel. Products must meet rules of origin requirements 
            to qualify for preferenti

KeyboardInterrupt: Interrupted by user